In [1]:
# connect to the server

from pprint import pprint
import json

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [2]:
with open("private.json", "r") as f:
    private = json.load(f)

uri = private["MONGO_ATLAS_URI"]

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi("1"))

# Send a ping to confirm a successful connection
try:
    client.admin.command("ping")
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [4]:
def read_county_data_file(filename):
    with open(filename, "r") as f:
        data = json.load(f)
    assert type(data) is dict
    assert data["type"] == "FeatureCollection"
    assert type(data["features"]) is list
    for feature in data["features"]:
        assert type(feature["properties"]["name"]) is str
        assert feature["geometry"]["type"] == "Polygon"
        assert type(feature["geometry"]["coordinates"]) is list
    return data


data = read_county_data_file("Franklin_County_Boundary.geojson")

In [6]:
ohio_db = client.ohio_db
ohio_db.drop_collection("boundaries")
boundaries_collection = ohio_db.boundaries


def load_county_data(filename, collection):
    data = read_county_data_file(filename)
    for feature in data["features"]:
        collection.insert_one(feature)
    print(f"Data from {filename} has been inserted into the collection")


load_county_data("Franklin_County_Boundary.geojson", boundaries_collection)
load_county_data("Cuyahoga_County_Boundary.geojson", boundaries_collection)
print("GeoJSON data has been inserted into the 'boundaries' collection.")

Data from Franklin_County_Boundary.geojson has been inserted into the collection
Data from Cuyahoga_County_Boundary.geojson has been inserted into the collection
GeoJSON data has been inserted into the 'boundaries' collection.
